In [1]:
import os
import pandas as pd
import glob
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook

In [18]:
def read_files(root_dir):
    files = glob.glob(os.path.join(root_dir, "*.csv"))
    data_frames = {}
    for idx, file in enumerate(files):
        with open(file, 'r', encoding='utf-8') as f:
            line1 = f.readline()            
            # Example line:
            # "Sitzungsdetail für 'Bodystyling mit Bibi':"
            course_name = f.readline().split("'")[1] # get the name of the course, e.g.: Bodystyling mit Bibi        
        df = pd.read_csv(file, sep="\t", skiprows=2)
        data_frames[idx] = (df, Path(file).stem.split('_')[0], course_name)
    return data_frames

In [19]:
def consolidate(df):
    df_cons = pd.DataFrame(columns = ['Name', 'E-Mail', 'Datum'])
    date = None
    groups = df.groupby('Name')
    for k,g in groups:
        for i, row in g.iterrows():
            df_cons = df_cons.append({'Name':row['Name'], 'E-Mail':row['E-Mail'], 'Datum':row['Datum']}, ignore_index=True)
            if date is None:
                date = row['Datum']
            break
    return (df_cons, date)

In [20]:
def save(df, path):
    df.to_csv(path,header=True, index=False, sep='\t', encoding='utf-8')

In [23]:
root_dir = r'C:\Users\rudi\Documents\Tina\WebexKurse\15.3-25.3'
data_frames = read_files(root_dir)
consolidate_data = []
for k,v in data_frames.items():
    df = v[0]
    name = v[1]
    (df_cons, date) = consolidate(df)
    filename = os.path.join(root_dir, f'{name}_{date}.tsv')
    save(df_cons, filename)